In [21]:
# Libraries and utilities

# RENINDER: FARE FIT.TRANSFORM SOLO SU TRAINING E DOPO SUL TEST SOLO .TRANSFORM
# REMINDER: NORMALIZZARE PRIMA DI FARE TUTTO IL PREPROCESSING 

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.model_selection import StratifiedKFold,RepeatedStratifiedKFold, RandomizedSearchCV,GridSearchCV, RepeatedKFold

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
)

In [22]:
df_train=pd.read_csv("./Dataset_prepared/Prepared_train.csv")
df_test=pd.read_csv("./Dataset_prepared/Prepared_test.csv")

In [23]:
df_train.head()

,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,filename,mean,q25,...,sc_skew_w4,stft_sum_w4,stft_q01_w4,stft_q05_w4,stft_q25_w4,stft_q50_w4,stft_q75_w4,stft_q95_w4,stft_kur_w4,stft_skew_w4
0,speech,neutral,normal,Kids are talking by the door,1st,1,M,03-01-01-01-01-01-01.wav,9.150000e-07,-0.000031,...,-4.175609,740.409892,0.000000,0.566462,0.709962,0.799141,0.896606,1.0,6.742190,-1.706215
1,speech,neutral,normal,Kids are talking by the door,2nd,1,M,03-01-01-01-01-02-01.wav,7.137790e-07,-0.000031,...,-1.100390,705.975006,0.368623,0.472736,0.623183,0.744908,0.874713,1.0,-0.700420,-0.201495
2,speech,neutral,normal,Dogs are sitting by the door,1st,1,M,03-01-01-01-02-01-01.wav,9.554485e-07,0.000000,...,-3.120218,694.365924,0.000000,0.417919,0.643636,0.774253,0.899156,1.0,1.688986,-1.024773
3,speech,neutral,normal,Dogs are sitting by the door,2nd,1,M,03-01-01-01-02-02-01.wav,9.176213e-07,-0.000031,...,-0.880988,663.205667,0.306280,0.399641,0.606910,0.755213,0.886474,1.0,-0.594111,-0.412871
4,speech,calm,normal,Kids are talking by the door,1st,1,M,03-01-02-01-01-01-01.wav,8.108948e-07,-0.000031,...,-1.071109,741.412796,0.248765,0.428202,0.634815,0.759914,0.878014,1.0,0.126535,-0.620782


In [24]:
df_train.drop('actor',axis=1,inplace=True)
df_train.drop('filename',axis=1,inplace=True)

df_test.drop('actor',axis=1,inplace=True)
df_test.drop('filename',axis=1,inplace=True)

In [25]:
le=preprocessing.LabelEncoder()
c = df_train.select_dtypes(include=['object']).columns.tolist()
for i in c:
    df_train[i]=le.fit_transform(df_train[i])

for i in c:
    df_test[i]=le.fit_transform(df_test[i])

Normalizzazione train e test

In [26]:
scaler = StandardScaler()
numeric_features = [n for n in df_train.columns if n!="emotion" and n!= "vocal_channel" and n!= "emotional_intensity" and n!= "statement" and n!= "repetition" and n!= "sex" and n!="filename" and n!="actor"]
scaled_features = scaler.fit_transform(df_train[numeric_features])
df_train[numeric_features] = scaled_features


scaled_features_test = scaler.transform(df_test[numeric_features])
df_test[numeric_features] = scaled_features_test



In [27]:
col=[x for x in df_train.columns if x!="emotion"]
X_train=df_train[col].values
y_train = np.array(df_train["emotion"])

In [28]:
X_test=df_test[col].values
y_test = np.array(df_test["emotion"])

DT

In [29]:
param_grid = {'max_depth': [None, 2, 5, 10, 15, 20],
              'min_samples_split': [2, 5, 10, 15, 20],
              'min_samples_leaf': [1, 5, 10, 15, 20]
}

grid = GridSearchCV(
    DecisionTreeClassifier(),
    param_grid=param_grid,
    cv=StratifiedKFold(), # with no parameters it is with 5 split and 10 repetitions
    refit=True,
    scoring="f1_macro"
)
grid.fit(X_train, y_train)
clf = grid.best_estimator_

In [30]:
print(grid.best_params_)

{'max_depth': 10, 'min_samples_leaf': 20, 'min_samples_split': 2}


In [31]:
learner = DecisionTreeClassifier(max_depth=10,min_samples_split=2,min_samples_leaf=20)
classifier = learner.fit(X_train, y_train)
predictions = classifier.predict(X_test)

print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.46      0.61      0.53        96
           1       0.47      0.33      0.39        96
           2       0.31      0.29      0.30        48
           3       0.26      0.21      0.23        96
           4       0.24      0.22      0.23        96
           5       0.29      0.48      0.36        48
           6       0.31      0.29      0.30        96
           7       0.44      0.44      0.44        48

    accuracy                           0.35       624
   macro avg       0.35      0.36      0.35       624
weighted avg       0.35      0.35      0.34       624



KNN

In [32]:
X_train=df_train[col].values
y_train = np.array(df_train["emotion"])

In [33]:
param_grid = {
    "n_neighbors": np.arange(2,51),
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "cityblock","minkowski"], # cityblock is the manhattan distance
}

grid = GridSearchCV(
    KNeighborsClassifier(),
    param_grid=param_grid,
    cv=StratifiedKFold(), # with no parameters it is with 5 split and 10 repetitions
    refit=True,
    scoring="f1_macro"
)

grid.fit(X_train, y_train)
clf = grid.best_estimator_

In [34]:
print(grid.best_params_)

{'metric': 'cityblock', 'n_neighbors': 40, 'weights': 'distance'}


In [60]:
X_test=df_test[col].values
y_test = np.array(df_test["emotion"])

In [35]:
learner = KNeighborsClassifier(n_neighbors= 40, metric= 'cityblock', weights= 'distance')
classifier = learner.fit(X_train, y_train)
predictions = classifier.predict(X_test)

print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.55      0.59      0.57        96
           1       0.49      0.56      0.52        96
           2       0.50      0.27      0.35        48
           3       0.45      0.39      0.41        96
           4       0.34      0.45      0.39        96
           5       0.42      0.27      0.33        48
           6       0.27      0.24      0.26        96
           7       0.28      0.35      0.31        48

    accuracy                           0.41       624
   macro avg       0.41      0.39      0.39       624
weighted avg       0.42      0.41      0.41       624

